# SPARQL in Ruby scripts

Executing a SPARQL query in Ruby is extremely simple. It uses the [SPARQL client](http://www.rubydoc.info/github/ruby-rdf/sparql-client/frames) Ruby Gem, which uses [RDF::Query::Solutions](http://www.rubydoc.info/github/ruby-rdf/rdf/RDF/Query/Solutions)and the code is as follows:



In [ ]:

require 'sparql/client'

endpoint = "http://sparql.uniprot.org/sparql"  # what location are we querying?

query = <<END
PREFIX foaf:<http://xmlns.com/foaf/0.1/>

SELECT ?name ?image                              # note that ?name and ?image becomes the Ruby symbol :name and :image
WHERE {
       ?taxon  <http://xmlns.com/foaf/0.1/depiction>  ?image .
       ?taxon <http://purl.uniprot.org/core/scientificName> ?name .
  FILTER regex(?name, '^Arabidopsis.*', 'i') .
}
END



sparql = SPARQL::Client.new(endpoint)  # create a SPARQL client
result = sparql.query(query)  # Execute query

puts result.class   # RDF::Query::Solutions (array of RDF::Query::Solution;  responds to #each

result.each do |solution|
  puts "Species #{solution[:name]}  is depicted by image #{solution[:image]}"  # call the pairs of variables in our query
end

puts ""


### and that's really all there is to it!  Easy!

<pre>


</pre>



# Loading, Creating and Writing RDF in Ruby

Here we will use the [<code>rdf</code> ruby gem](http://ruby-rdf.github.io/rdf/) which is extremely easy to understand.  There are two ways to work with RDF in Ruby: 

### Method 1
You can start with an RDF file (either locally or on the Web), and load it into a "graph" as follows:



In [ ]:
require 'rdf'
require 'rdf/ntriples'

graph = RDF::Graph.load("http://ruby-rdf.github.com/rdf/etc/doap.nt")
puts graph.class  # RDF::Graph
puts graph.size   # number of triples in the graph




(note that we will ONLY use ntriples in this course - a more common RDF format is 'Turtle' but there are efficiencies we can gain by using ntriples format, so that is what we will use.  You can read the documentation for yourself to learn how to load other RDF formats...)

 - once we have loaded the data, we convert it into an RDF::Repository Object, and then use that exactly as we used the SPARQL endpoint in the example above:
 
 

In [ ]:
require 'rdf'
require 'sparql/client'
require 'rdf/ntriples'
require 'rdf/repository'


graph = RDF::Graph.load("http://ruby-rdf.github.com/rdf/etc/doap.nt")
puts graph.class  # RDF::Graph
puts graph.size   # number of triples in the graph




repo = RDF::Repository.new   # a repository can be used as a SPARQL endpoint for SPARQL::Client
repo.insert(*graph)   # the "splat" operator (*) - converts an array into a list of individual things
                      # in this case, the individual triples in the RDF::Graph
puts repo.count

query = <<END
PREFIX foaf:<http://xmlns.com/foaf/0.1/>

SELECT ?name
WHERE {
       ?p a foaf:Person .
       ?p foaf:name ?name
}
END

sparql2 = SPARQL::Client.new(repo)    # Exactly the same as above...
result = sparql2.query(query)  # Execute query

result.each do |solution|
  puts "Database contains the person:  #{solution[:name]}"  # call the pairs of variables in our query
end

puts ""


### Method 2
Another way is to create the RDF _de novo_.  For this, we will again use the rdf Gem, and in particular, the [RDF::Statement](http://www.rubydoc.info/github/ruby-rdf/rdf/master/RDF/Statement) Class.

It is useful to know some common ontologys for predicates:

| Ontology | URL |
| --- | --- |
| Dublin Core | http://purl.org/dc/terms/ |
| Friend of a Friend (FOAF) |  http://xmlns.com/foaf/0.1/knows |
| RDF | http://www.w3.org/1999/02/22-rdf-syntax-ns |
| RDF Schema (RDFS) | http://www.w3.org/2000/01/rdf-schema |
| Web Ontology Language (OWL) | http://www.w3.org/2002/07/owl# |
| XML Schema Datatypes (XSD) | http://www.w3.org/2001/XMLSchema# |

(note that you can use ANY of the terms in those ontologies... for example DC.author or FOAF.name)


Here is an example of the code:



In [11]:
require 'rdf'
require 'sparql/client'
require 'rdf/ntriples'
require 'rdf/repository'


repo = RDF::Repository.new   # a repository can be used as a SPARQL endpoint for SPARQL::Client

# create the vocabularies I will use (this is optional, it just makes the code easier to read later)
foaf = RDF::Vocabulary.new("http://xmlns.com/foaf/0.1/")
dc = RDF::Vocabulary.new("http://purl.org/dc/terms/")
rdf = RDF::Vocabulary.new("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

# ---------------   READY!

# create the Subject, Predicate, and Object of a triple
s = RDF::URI.new("http://wilkinsonlab.info/Courses")
p = dc.creator   # some predicates are pre-defined by the rdf libraries...  see below
o = RDF::URI.new("http://wilkinsonlab.info/#mark")
triple = RDF::Statement(s, p, o) # make a statement object (a triple)
repo.insert(triple)  # add it to your repository


# create the Subject, Predicate, and Object of the next triple
s = RDF::URI.new("http://wilkinsonlab.info/#mark")
p = foaf.name
o = RDF::Literal.new("Mark Wilkinson")
triple = RDF::Statement(s, p, o) # make a statement object (a triple)
repo.insert(triple)  # add it to your repository


# create the Subject, Predicate, and Object of the last triple
s = RDF::URI.new("http://wilkinsonlab.info/#mark")
p = rdf.type
o = foaf.Person
triple = RDF::Statement(s, p, o) # make a statement object (a triple)
repo.insert(triple)  # add it to your repository


# and from here, it is exactly the same as before...

query = <<END
PREFIX foaf:<http://xmlns.com/foaf/0.1/>

SELECT ?name
WHERE {
       ?p a foaf:Person .
       ?p foaf:name ?name
}
END

sparql2 = SPARQL::Client.new(repo)    # Exactly the same as above...
result = sparql2.query(query)  # Execute query

result.each do |solution|
  puts "Database contains the person:  #{solution[:name]}"  # call the pairs of variables in our query
end

puts "done"

Database contains the person:  Mark Wilkinson
done


# Writing RDF to a file

The final thing we need to do is export our beautiful RDF to a file so that it can be loaded into a database.  This is easy too:

**(look in your Lectures/files/ folder to see the output...)**



In [14]:
RDF::Writer.open("./files/MarkWilkinson.nt") { |writer| writer << repo }


#<RDF::NTriples::Writer:0x00005632705cbbd0 @options={:encoding=>#<Encoding:UTF-8>, :validate=>true, :logger=>#<IRuby::OStream:0x000056326f2cf248 @session=#<IRuby::Session:0x000056326f2cfc48 @sockets={:publish=>#<CZTop::Socket::PUB:0x56326f2d3450 last_endpoint="tcp://127.0.0.1:50755">, :reply=>#<CZTop::Socket::ROUTER:0x56326f2c9e30 last_endpoint="tcp://127.0.0.1:41617">, :stdin=>#<CZTop::Socket::ROUTER:0x56326f2d6210 last_endpoint="tcp://127.0.0.1:57399">}, @session="60c7a93d-641c-42c4-be58-2bc139595fb2", @hmac=f429c5d83b6a471970abebea39e3875ca460aa12de315bfc15ca851b4b840b22, @last_recvd_msg={:idents=>["B1BC599411D74A2D8E3823F814905077"], :header=>{"msg_id"=>"F52F1A9A2CFA445786E698054A3DA77C", "username"=>"username", "session"=>"B1BC599411D74A2D8E3823F814905077", "msg_type"=>"execute_request", "version"=>"5.0", "date"=>"2017-12-19T12:34:28.772033Z"}, :parent_header=>{}, :metadata=>{}, :content=>{"code"=>"RDF::Writer.open(\"./files/MarkWilkinson.nt\") { |writer| writer << repo }\n", "sil


# THAT's ALL!
